In [4]:
!pip install langgraph
!pip install -U duckduckgo-search

  Using cached duckduckgo_search-6.2.11-py3-none-any.whl.metadata (24 kB)
  Using cached primp-0.6.1-cp38-abi3-macosx_11_0_arm64.whl.metadata (10 kB)
Using cached duckduckgo_search-6.2.11-py3-none-any.whl (27 kB)
Using cached primp-0.6.1-cp38-abi3-macosx_11_0_arm64.whl (2.5 MB)


In [16]:
from langgraph.graph import Graph
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from typing import TypedDict, Annotated, Sequence

# 상태 정의
class State(TypedDict):
    messages: Annotated[Sequence[HumanMessage], "대화 기록"]
    budget: Annotated[float, "예산"]
    date: Annotated[str, "결혼 날짜"]

# 웨딩 플래너 노드 정의
def wedding_planner(state: State) -> State:
    # 웨딩 플래너 로직 구현
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 전문 웨딩 플래너입니다. 고객의 예산과 날짜를 고려하여 조언을 제공하세요."),
        ("human", "{input}")
    ])
    model = ChatOpenAI()
    response = model.invoke(prompt.format_messages(input=f"예산: {state['budget']}, 날짜: {state['date']}"))
    state["messages"].append(HumanMessage(content=response.content))
    return state

# 예산 관리 노드 정의
def budget_manager(state: State) -> State:
    # 예산 관리 로직 구현
    # ...
    return state

# 날짜 관리 노드 정의
def date_manager(state: State) -> State:
    # 날짜 관리 로직 구현
    # ...
    return state

# 그래프 생성
workflow = StateGraph(State)

# 노드 추가
workflow.add_node("wedding_planner", wedding_planner)
workflow.add_node("budget_manager", budget_manager)
workflow.add_node("date_manager", date_manager)

# 엣지 추가
workflow.add_edge("wedding_planner", "budget_manager")
workflow.add_edge("budget_manager", "date_manager")
workflow.add_edge("date_manager", "wedding_planner")

# 그래프 컴파일
app = workflow.compile()

# 초기 상태 설정
initial_state = State(messages=[], budget=10000000, date="2023-12-25")

# 그래프 실행
for state in app.stream(initial_state):
    print(state)

In [9]:
from langgraph.graph import Graph
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.schema import HumanMessage
from typing import TypedDict, Annotated, Sequence
import sqlite3

# 상태 정의
class State(TypedDict):
    messages: Annotated[Sequence[HumanMessage], "대화 기록"]
    budget: Annotated[float, "예산"]
    date: Annotated[str, "결혼 날짜"]
    preferences: Annotated[dict, "고객 취향"]
    selected_vendors: Annotated[list, "선정된 업체 목록"]

# 웹 검색 도구 초기화
search = DuckDuckGoSearchRun()

# 데이터베이스 연결
conn = sqlite3.connect('wedding_vendors.db')
cursor = conn.cursor()

# 업체 테이블 생성 (필요한 경우)
cursor.execute('''
CREATE TABLE IF NOT EXISTS vendors (
    name TEXT,
    style TEXT,
    price REAL
)
''')
conn.commit()

# 가짜 데이터 삽입
fake_vendors = [
    ("Modern Wedding Studio", "모던", 5000000),
    ("Classic Elegance", "클래식", 7000000),
    ("Rustic Charm", "러스틱", 3000000),
    ("Urban Chic Events", "모던", 8000000),
    ("Luxury Weddings", "럭셔리", 15000000),
]

# 기존 데이터를 삭제하고 가짜 데이터 삽입
cursor.execute('DELETE FROM vendors')
cursor.executemany('INSERT INTO vendors (name, style, price) VALUES (?, ?, ?)', fake_vendors)
conn.commit()

# 업체 검색 및 선정 노드
def vendor_selection(state: State) -> State:
    # 고객 취향과 예산을 기반으로 웹 검색 수행
    query = f"웨딩 업체 {state['preferences']['style']} 스타일 예산 {state['budget']}"
    search_results = search.run(query)

    # 데이터베이스에서 조건에 맞는 업체 검색
    cursor.execute("""
        SELECT name, price FROM vendors 
        WHERE style = ? AND price <= ?
    """, (state['preferences']['style'], state['budget']))
    db_results = cursor.fetchall()

    # 검색 결과와 데이터베이스 결과를 종합하여 업체 선정
    selected_vendors = []
    for vendor in db_results:
        if any(vendor[0] in result for result in search_results):
            selected_vendors.append(vendor)

    # 데이터베이스 결과만 사용하여 선정
    if not selected_vendors:  # 만약 검색 결과가 없다면
        selected_vendors = db_results

    state['selected_vendors'] = selected_vendors
    return state

# 예산 확정 노드
def budget_confirmation(state: State) -> State:
    total_budget = sum(vendor[1] for vendor in state['selected_vendors'])
    state['budget'] = total_budget

    # 예산 확정 메시지 추가
    confirmation_message = f"선정된 업체들의 총 예산은 {total_budget}원입니다."
    state['messages'].append(HumanMessage(content=confirmation_message))

    return state

# 웨딩 플래너 노드
def wedding_planner(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 전문 웨딩 플래너입니다. 선정된 업체와 확정된 예산을 바탕으로 조언을 제공하세요."),
        ("human", "{input}")
    ])
    model = ChatOpenAI()
    response = model.invoke(prompt.format_messages(
        input=f"선정된 업체: {state['selected_vendors']}, 확정 예산: {state['budget']}"
    ))
    state["messages"].append(HumanMessage(content=response.content))
    return state

# LangGraph 생성
workflow = Graph()

# 노드 추가
workflow.add_node("vendor_selection", vendor_selection)
workflow.add_node("budget_confirmation", budget_confirmation)
workflow.add_node("wedding_planner", wedding_planner)

# 엣지 추가 (노드 간 연결)
workflow.add_edge("vendor_selection", "budget_confirmation")
workflow.add_edge("budget_confirmation", "wedding_planner")

# 그래프 컴파일
app = workflow.compile()

# 초기 상태 설정
initial_state = State(
    messages=[],
    budget=10000000,  # 예산 설정
    date="2023-12-25",  # 결혼 날짜 설정
    preferences={"style": "모던"},  # 고객 취향 설정
    selected_vendors=[]
)

# 그래프 실행
for state in app.stream(initial_state, {"input": "vendor_selection"}):
    print(state)

ValueError: Node `vendor_selection` is not reachable

In [17]:
# LangGraph 구조 정의
from langgraph.graph import Graph

# 노드 함수 정의 (실제 구현은 생략)
def customer_input(state):
    # 고객 정보 입력 로직
    return state

def budget_setting(state):
    # 예산 설정 로직
    return state

def vendor_selection(state):
    # 업체 검색 및 선정 로직
    return state

def budget_adjustment(state):
    # 예산 조정 로직
    return state

def final_planning(state):
    # 최종 계획 수립 로직
    return state

# 그래프 생성
workflow = Graph()

# 노드 추가
workflow.add_node("customer_input", customer_input)
workflow.add_node("budget_setting", budget_setting)
workflow.add_node("vendor_selection", vendor_selection)
workflow.add_node("budget_adjustment", budget_adjustment)
workflow.add_node("final_planning", final_planning)

# 엣지 추가
workflow.add_edge("customer_input", "budget_setting")
workflow.add_edge("budget_setting", "vendor_selection")
workflow.add_edge("vendor_selection", "budget_adjustment")
workflow.add_edge("budget_adjustment", "final_planning")
workflow.add_edge("budget_adjustment", "vendor_selection")  # 예산 조정 후 재검색 가능

# 그래프 컴파일
app = workflow.compile()

ValueError: Already found path for node 'budget_adjustment'.
For multiple edges, use StateGraph with an Annotated state key.